<a href="https://colab.research.google.com/github/alexlaurence/DoReMi/blob/master/DoReMi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DoReMi (v0.1.6-alpha)

![alt text](https://i.imgur.com/OS76n8F.png)

By Alexander Laurence (Email: alexander.adamlaurence@gmail.com | Web: http://celestial.tokyo/~AlexLaurence/)

*DoReMi is a tool for blind users to search job listings from by synthesising job listing RSS feeds from Scout.co.kr into speech with the help of Google's Text-to-Speech API powered by a WaveNet model.*

**Why Google Text-to-Speech?**

The Text-to-Speech API creates raw audio data of natural, human speech. That is, it creates audio that sounds like a person talking. When you send a synthesis request to the Text-to-Speech API, you must specify a voice that 'speaks' the words.

**Why WaveNet?**

A WaveNet generates speech that sounds more natural than other text-to-speech systems. It synthesizes speech with more human-like emphasis and inflection on syllables, phonemes, and words. On average, a WaveNet produces speech audio that people prefer over other text-to-speech technologies.

Unlike most other text-to-speech systems, a WaveNet model creates raw audio waveforms from scratch. The model uses a neural network that has been trained using a large volume of speech samples. During training, the network extracts the underlying structure of the speech, such as which tones follow each other and what a realistic speech waveform looks like. When given a text input, the trained WaveNet model can generate the corresponding speech waveforms from scratch, one sample at a time, with up to 24,000 samples per second and seamless transitions between the individual sounds.

## Installation

In [0]:
#@title 
!pip install feedparser
!pip install google-cloud-texttospeech

import feedparser
import re, os, glob, shutil

from google.cloud import texttospeech
from google.oauth2 import service_account

from IPython.display import Audio
from scipy.io import wavfile

ask_cred = input("Please enter the path of your Google Cloud Credential File: ")

#set API Credentials
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]=ask_cred

# Try It Out

In [4]:
def start():
  
  #@markdown **Audio Settings**:
  Voice = "ko-KR-Wavenet-D" #@param ["ko-KR-Wavenet-A", "ko-KR-Wavenet-B", "ko-KR-Wavenet-C", "ko-KR-Wavenet-D", "ko-KR-Standard-A", "ko-KR-Standard-B", "ko-KR-Standard-C", "ko-KR-Standard-D"]
  Speed = 0.65 #@param {type:"slider", min:0, max:1, step:0.05}

  #@markdown --------------------------------------

  #@markdown **DoReMi (v0.1.0-alpha)**

  
  credentials = service_account.Credentials.from_service_account_file(ask_cred)
  client = texttospeech.TextToSpeechClient(credentials=credentials)
  
  # set xml url as strings
  seoul = "http://feeds.feedburner.com/co/clsY"
  gangwon = "http://feeds.feedburner.com/co/Utav"
  gyeonggi = "http://feeds.feedburner.com/co/lUKP"
  gyeongnam = "http://feeds.feedburner.com/co/rMCwN"
  gyeongbuk = "http://feeds.feedburner.com/co/Dhwm"
  gwangju = "http://feeds.feedburner.com/co/LXwe"
  daegu = "http://feeds.feedburner.com/co/Bcsg"
  daejeon = "http://feeds.feedburner.com/co/zBIF"
  busan = "http://feeds.feedburner.com/co/dkiFn"
  ulsan = "http://feeds.feedburner.com/co/LFJz"
  incheon = "http://feeds.feedburner.com/co/EkCu"
  jeonnam = "http://feeds.feedburner.com/co/wjlNw"
  jeonbuk = "http://feeds.feedburner.com/co/ZaXx"
  jeju = "http://feeds.feedburner.com/co/viTpC"
  chungnam = "http://feeds.feedburner.com/co/yIBb"
  chungbuk = "http://feeds.feedburner.com/co/aXuFR"
  national = "http://feeds.feedburner.com/co/TmxG"

  # ask user to enter the city
  get_city = input('도시 이름: ').lower()
  print('OK! {}. 잠시만 기다려주세요...'.format(get_city))
  
  # set the value to the corresponding xml url
  if get_city == '서울' or get_city == 'seoul':
    get_city = seoul
  elif get_city == '강원' or get_city == 'gangwon':
    get_city = gangwon
  elif get_city == '경기' or get_city == 'gyeonggi':
    get_city = gyeonggi
  elif get_city == '경남' or get_city == 'gyeongnam':
    get_city = gyeongnam
  elif get_city == '경북' or get_city == 'gyeongbuk':
    get_city = gyeongbuk
  elif get_city == '광주' or get_city == 'gwangju':
    get_city = gwangju
  elif get_city == '대구' or get_city == 'daegu':
    get_city = daegu
  elif get_city == '대전' or get_city == 'daejeon':
    get_city = daejeon
  elif get_city == '부산' or get_city == 'busan':
    get_city = busan
  elif get_city == '울산' or get_city == 'ulsan':
    get_city = ulsan
  elif get_city == '인천' or get_city == 'incheon':
    get_city = incheon
  elif get_city == '전남' or get_city == 'jeonnam':
    get_city = jeonnam
  elif get_city == '전북' or get_city == 'jeonbuk':
    get_city = jeonbuk
  elif get_city == '제주' or get_city == 'jeju':
    get_city = jeju
  elif get_city == '충남' or get_city == 'chungnam':
    get_city = chungnam
  elif get_city == '충북' or get_city == 'chungbuk':
    get_city = chungbuk
  elif get_city == '전국' or get_city == 'national':
    get_city = national
  else:
    print('미안 난 이해를 못 했어...')
    start()

  # parse RSS feed
  feed = feedparser.parse(get_city)
  feed_title = feed['feed']['title']
  feed_entries = feed.entries
  
  # set voice settings
  voice = texttospeech.types.VoiceSelectionParams(
      language_code='ko-KR',
      name=Voice,
      ssml_gender=texttospeech.enums.SsmlVoiceGender.FEMALE)

  audio_config = texttospeech.types.AudioConfig(
      audio_encoding=texttospeech.enums.AudioEncoding.MP3,
      speaking_rate=Speed)
  
  # Create directory
  dst = '/content/audio/'
  if not os.path.exists(dst):
    os.makedirs(dst)

  # read entries
  i=1
  for entry in feed.entries:
    article_title = entry.title
    article_description = entry.description
    article_link = entry.link
    
    clean_title = re.sub("-", "", article_title)
    clean_title2 = re.sub("amp;", "", clean_title)

    clean_description = re.sub("<.*?>", "", article_description)
    clean_description2 = re.sub(":", "", clean_description)
    clean_description3 = re.sub("·", " ", clean_description2)
    clean_description4 = re.sub("-", "", clean_description3)
    clean_description5 = re.sub("amp;", "", clean_description4)
    
    # synthesise text to speech
    input_text = texttospeech.types.SynthesisInput(text=clean_title2+" "+clean_description5)
    response = client.synthesize_speech(input_text, voice, audio_config)
    
    # The response's audio_content is binary.
    with open('listing_{}.mp3'.format(i), 'wb') as out:
        out.write(response.audio_content)
        print('\n')
        display(Audio('listing_{}.mp3'.format(i),rate=44100))
        
        src = "/content/"
        shutil.move(os.path.join(src, "listing_{}.mp3".format(i)), os.path.join(dst, "listing_{}.mp3".format(i)))
        #print('Audio content written to file "output.mp3"')
        
    print("\n {} {}".format(clean_title2, clean_description5))
    print("\n{}".format(article_link))
                
    i=i+1
    
print('DoReMi에서 오신 것을 환영합니다!')
start()

DoReMi에서 오신 것을 환영합니다!
도시 이름: seoul
OK! seoul. 잠시만 기다려주세요...





 크리스패션  (주)크리스F&C  파리게이츠/마스터바니 영업MD 경력직 채용  회사명  크리스패션 모집내용  크리스패션  (주)크리스F&C  파리게이츠/마스터바니 영업MD 경력직 채용  직종   근무형태  정규직 경력  경력 학력  전졸(예) 지역  서울 마감일  채용시까지 입사지원하기 크리스패션 기업의 진행중인 채용정보 보기

http://www.scout.co.kr/jobs/all/contentview.asp?id=1594819&page=1





 렙쇼메이  메종블랑쉬  영업MD 경력 모집  회사명  렙쇼메이 모집내용  렙쇼메이  메종블랑쉬  영업MD 경력 모집 직종   근무형태  정규직, 계약직 경력  경력 학력  고졸(예) 지역  서울 마감일  06/20(목) 입사지원하기 렙쇼메이 기업의 진행중인 채용정보 보기

http://www.scout.co.kr/jobs/all/contentview.asp?id=1594602&page=1





 (주)린컴퍼니  린컴퍼니(케네스레이디)와 함께 할 디자이너(팀장급)를 모십니다  회사명  (주)린컴퍼니 모집내용  (주)린컴퍼니  린컴퍼니(케네스레이디)와 함께 할 디자이너(팀장급)를 모십니다 직종   근무형태  정규직 경력  경력 학력  전졸(예) 지역  서울 마감일  05/30(목) 입사지원하기 (주)린컴퍼니 기업의 진행중인 채용정보 보기

http://www.scout.co.kr/jobs/all/contentview.asp?id=1592678&page=1





 (주)인동에프엔  쉬즈미스/리스트 기획MD, 기획관리(원자재) 신입 및 경력 채용  회사명  (주)인동에프엔 모집내용  (주)인동에프엔  쉬즈미스/리스트 기획MD, 기획관리(원자재) 신입 및 경력 채용 직종   근무형태  정규직 경력  무관 학력  대졸(예) 지역  서울 마감일  채용시까지 입사지원하기 (주)인동에프엔 기업의 진행중인 채용정보 보기

http://www.scout.co.kr/jobs/all/contentview.asp?id=1595076&page=1




...


 (주)인동에프엔  쉬즈미스/리스트 생산관리, 부자재구매, 니트MR, 패턴 경력 채용  회사명  (주)인동에프엔 모집내용  (주)인동에프엔  쉬즈미스/리스트 생산관리, 부자재구매, 니트MR, 패턴 경력 채용 직종   근무형태  정규직 경력  경력 학력  대졸(예) 지역  서울 마감일  채용시까지 입사지원하기 (주)인동에프엔 기업의 진행중인 채용정보 보기

http://www.scout.co.kr/jobs/all/contentview.asp?id=1595072&page=1





 (주)인동에프엔  쉬즈미스/리스트 마케팅, VMD 경력 채용  회사명  (주)인동에프엔 모집내용  (주)인동에프엔  쉬즈미스/리스트 마케팅, VMD 경력 채용 직종   근무형태  정규직 경력  경력 학력  대졸(예) 지역  서울 마감일  채용시까지 입사지원하기 (주)인동에프엔 기업의 진행중인 채용정보 보기

http://www.scout.co.kr/jobs/all/contentview.asp?id=1595074&page=1





 (주)인동에프엔  쉬즈미스/리스트 디자이너 각 부문 인턴 신입 및 경력 채용  회사명  (주)인동에프엔 모집내용  (주)인동에프엔  쉬즈미스/리스트 디자이너 각 부문 인턴 신입 및 경력 채용 직종   근무형태  정규직, 인턴 경력  무관 학력  대졸(예) 지역  서울 마감일  채용시까지 입사지원하기 (주)인동에프엔 기업의 진행중인 채용정보 보기

http://www.scout.co.kr/jobs/all/contentview.asp?id=1595070&page=1





 애경산업(주)  2019년 애경산업 계약직 직원 채용(영업교육&기획파트)  회사명  애경산업(주) 모집내용  애경산업(주)  2019년 애경산업 계약직 직원 채용(영업교육&기획파트) 직종   근무형태  계약직 경력  무관 학력  대졸(예) 지역  서울 마감일  05/29(수) 입사지원하기 애경산업(주) 기업의 진행중인 채용정보 보기

http://www.scout.co.kr/jobs/all/contentview.asp?id=1595137&page=1





 서울대학교  법학발전재단 직원 채용  회사명  서울대학교 모집내용  서울대학교  법학발전재단 직원 채용 직종   근무형태  계약직 경력  무관 학력  무관 지역  서울 마감일  06/07(금) 입사지원하기 서울대학교 기업의 진행중인 채용정보 보기

http://www.scout.co.kr/jobs/all/contentview.asp?id=1595136&page=1
